In [25]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown
from ordered_set import OrderedSet
import string

In [26]:
base_df = pd.read_csv('file/base_df3.csv')
fm_base_df = pd.read_csv('file/fm_base_df3.csv')
f_base_df = pd.read_csv('file/f_base_df3.csv')
sc_base_df = pd.read_csv('file/sc_base_df3.csv')
dr_base_df = pd.read_csv('file/dr_base_df3.csv')

In [27]:
def return_multiple_author(base_df):
    author = base_df['author'].tolist()
    author =\
    [i.replace('[Provider:earticle]','') \
     if type(i) != float and '[provider:earticle]' in i.lower() else i for i in author]
    base_df['journal'] = author
    return base_df

base_df = return_multiple_author(base_df)
fm_base_df = return_multiple_author(fm_base_df)
f_base_df = return_multiple_author(f_base_df)
sc_base_df = return_multiple_author(sc_base_df)
dr_base_df = return_multiple_author(dr_base_df)

In [28]:
base_df['year'] = base_df['year'].apply(lambda x : str(x).split('.')[0])
fm_base_df['year'] = fm_base_df['year'].apply(lambda x : str(x).split('.')[0])
f_base_df['year'] = f_base_df['year'].apply(lambda x : str(x).split('.')[0])
sc_base_df['year'] = sc_base_df['year'].apply(lambda x : str(x).split('.')[0])
dr_base_df['year'] = dr_base_df['year'].apply(lambda x : str(x).split('.')[0])

In [29]:
counter_df = pd.DataFrame.from_dict(Counter(base_df['author']),orient='index',columns=['count']).sort_values('count',ascending=False)
counter_df.head(10)

,count
Fama E. F. and K. R. French,463
Jensen M. and W. Meckling,186
Black F.,163
Carhart Mark M.,138
Fama E. F. and J. D. MacBeth,126
Jegadeesh N. and S. Titman,114
Jensen M. C.,108
Black F. and M. Scholes,107
Myers S. and N. Majluf,94
Newey W. K. and K. D. West,82


In [30]:
catch_author_ls = counter_df[counter_df['count'] > 3].index.tolist()
print(len(catch_author_ls))
catch_author_ls[:5]

1407


['Fama E. F. and K. R. French ',
 'Jensen M. and W. Meckling ',
 'Black F. ',
 'Carhart Mark M. ',
 'Fama E. F. and J. D. MacBeth ']

In [31]:
catch_author_ls_copy = catch_author_ls.copy()
tuned_catch_author_ls = []
excepted_ls = []
similar_match_ls = []

for i in range(5) : 
    print(i)
    for idx1,val1 in enumerate(catch_author_ls_copy) : 
        for idx2,val2 in enumerate(catch_author_ls_copy) :
            if type(val1) != float and type(val2) != float : 
                if idx1 >= idx2 : 
                    v1 = set(val1.split(' '))
                    v2 = set(val2.split(' '))
                    try : 
                        if len(set(v1).intersection(v2)) / ((len(v1) + len(v2))/2) > 0.85 : 
                            similar_match_ls.append((val1,val2))
                            if len(v1) > len(v2) :
                                catch_author_ls_copy.pop(catch_author_ls_copy.index(val2))
                                excepted_ls.append((val2))
                            else :
                                catch_author_ls_copy.pop(catch_author_ls_copy.index(val1))
                                excepted_ls.append(((val1)))                    
                    except : pass

len(catch_author_ls) , len(catch_author_ls_copy) , len(excepted_ls)

0
1
2
3
4


(1407, 44, 1363)

In [32]:
ready_to_target_dict = dict(zip(catch_author_ls_copy,[[] for i in range(len(catch_author_ls_copy))]))
append = 0
naive_append = 0

for idx1,val1 in enumerate(base_df['author'].tolist()) : 
    if idx1 % 4000 == 0 : print(idx1,end='\r')
    for idx2,val2 in enumerate(ready_to_target_dict.keys()) :
        
        if type(val1) != float and type(val2) != float:
            paper_name = val1.lower()
            target_name = val2.lower()

            paper_name_to_word = set(paper_name.split(' '))
            target_name_to_word = set(target_name.split(' '))
            if len(paper_name_to_word.intersection(target_name_to_word)) / \
            ((len(paper_name_to_word) + len(target_name_to_word))/2) > 0.85 : 
                ready_to_target_dict[val2].append(val1)
                append += 1

In [33]:
pickle_out = open("author_ready_to_target_dict.pickle","wb")
pickle.dump(ready_to_target_dict, pickle_out)
pickle_out.close()

외자가 한 칸 뛰어져 있는 경우를 붙여주어, 분산되는 count를 하나로 묶어 준다.

In [34]:
def convert_single_last_name(df) :
    df.reset_index(drop=True,inplace=True)
    single_first_name_dict = {'최 혁' : '최혁',
    '김 솔' : '김솔',
    '김 범' : '김범',
    '장 욱' : '장욱',
    '강 원' : '강원',
    '유 진' : '유진'}
    author_ls = []

    for name in list(single_first_name_dict.keys()) : 
        for idx,author in enumerate(df['author']) : 
            if type(author) != float : 
                if name in author : 
                    df['author'][idx] = re.sub(name , single_first_name_dict.get(name) , df['author'][idx])
    return df

In [35]:
base_df = convert_single_last_name(base_df)
fm_base_df = convert_single_last_name(fm_base_df)
f_base_df = convert_single_last_name(f_base_df)
sc_base_df = convert_single_last_name(sc_base_df)
dr_base_df = convert_single_last_name(dr_base_df)

아래에서부터는 multiple_and_single_author notation 을 only_single_author notation으로 split해주는 프로세스이다.

In [36]:
def parse_multiple_to_single(author) :
    if type(author) == str :
        author = author.strip()
        if re.search('[가-힣]',author) : 
            return re.split(' ',author)
        elif re.search('[a-zA-Z]',author) : 
            return re.split('and',author)
    else : return author

In [37]:
def return_df_author_frequency_table_under_journal(base_df):
    final_ls = []
    process_ls = []

    for idx1 , val in enumerate(base_df['author']) : 
        process_ls = []
        for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
            if val in ready_to_target_dict.get(key_) : 
                process_ls.append(key_)
        if type(val) == str :
            if len(process_ls) == 1 :
                final_ls.append(process_ls[0])
            elif len(process_ls) == 0 :
                final_ls.append(val)
            else : final_ls.append(val)
        else : final_ls.append(None)

    if len(final_ls) == len(base_df) : 
        base_df['new_author'] = final_ls
        del base_df['author']
    else : print('error')
    
    print("lower uniqueness means better performance, orgin uniqueness : {} , revised uniqueness : {}"\
          .format(len(set(base_df['new_author'])),len(set(final_ls))))
    return base_df

In [49]:
base_final_df = return_df_author_frequency_table_under_journal(base_df)
fm_final_df = return_df_author_frequency_table_under_journal(fm_base_df)
f_final_df = return_df_author_frequency_table_under_journal(f_base_df)
sc_final_df = return_df_author_frequency_table_under_journal(sc_base_df)
dr_final_df = return_df_author_frequency_table_under_journal(dr_base_df)

lower uniqueness means better performance, orgin uniqueness : 11472 , revised uniqueness : 11472
lower uniqueness means better performance, orgin uniqueness : 6479 , revised uniqueness : 6479
lower uniqueness means better performance, orgin uniqueness : 4592 , revised uniqueness : 4592
lower uniqueness means better performance, orgin uniqueness : 2711 , revised uniqueness : 2711
lower uniqueness means better performance, orgin uniqueness : 1847 , revised uniqueness : 1847


In [39]:
base_final_df['single_author'] = base_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
fm_final_df['single_author'] = fm_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
f_final_df['single_author'] = f_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
sc_final_df['single_author'] = sc_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))
dr_final_df['single_author'] = dr_final_df['new_author'].apply(lambda x : parse_multiple_to_single(x))

In [40]:
def make_paper_and_author_column(df) : 
    new_col = []
    for idx in range(len(df)) : 
        try :             
            new_col.append(df['tuned_new_paper'][idx] + '\\\\' + df['new_author'][idx])
        except : 
            new_col.append(df['tuned_new_paper'][idx] + '\\\\' + 'None_Value')
            
    df['paper_and_year_and_author'] = new_col
    return df

In [41]:
base_final_df = make_paper_and_author_column(base_final_df)
fm_final_df = make_paper_and_author_column(fm_final_df)
f_final_df = make_paper_and_author_column(f_final_df)
sc_final_df = make_paper_and_author_column(sc_final_df)
dr_final_df = make_paper_and_author_column(dr_final_df)

In [42]:
base_final_df.to_csv("file/base_df4.csv",index=False)
fm_final_df.to_csv("file/fm_base_df4.csv",index=False)
f_final_df.to_csv("file/f_base_df4.csv",index=False)
sc_final_df.to_csv("file/sc_base_df4.csv",index=False)
dr_final_df.to_csv("file/dr_base_df4.csv",index=False)

In [43]:
base_testing_df = base_final_df[base_final_df['single_author'].notnull()]
fm_testing_df = fm_final_df[fm_final_df['single_author'].notnull()]
f_testing_df = f_final_df[f_final_df['single_author'].notnull()]
sc_testing_df = sc_final_df[sc_final_df['single_author'].notnull()]
dr_testing_df = dr_final_df[dr_final_df['single_author'].notnull()]

In [44]:
base_testing_ls = [i for i in base_testing_df['single_author'] if len(i) < 5]
fm_testing_ls = [i for i in fm_testing_df['single_author'] if len(i) < 5]
f_testing_ls = [i for i in f_testing_df['single_author'] if len(i) < 5]
sc_testing_ls = [i for i in sc_testing_df['single_author'] if len(i) < 5]
dr_testing_ls = [i for i in dr_testing_df['single_author'] if len(i) < 5]

In [45]:
base_single_author_ls = [j.strip() for i in base_testing_ls for j in i if j]
fm_single_author_ls = [j.strip() for i in fm_testing_ls for j in i if j]
f_single_author_ls = [j.strip() for i in f_testing_ls for j in i if j]
sc_single_author_ls = [j.strip() for i in sc_testing_ls for j in i if j]
dr_single_author_ls = [j.strip() for i in dr_testing_ls for j in i if j]

In [46]:
author_counter_df = pd.DataFrame.from_dict(Counter(base_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False)
author_counter_df.reset_index(inplace=True)
display(author_counter_df.head(10))
author_counter_df.to_csv('author_counter_df.csv',index=False)

,index,count
0,Fama E. F.,638
1,K. R. French,465
2,Black F.,279
3,Jensen M.,193
4,S. Titman,187
5,W. Meckling,186
6,R. Vishny,164
7,고봉찬,153
8,최혁,141
9,Carhart Mark M.,138


In [47]:
display(pd.DataFrame.from_dict(Counter(fm_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(f_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(sc_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))
display(pd.DataFrame.from_dict(Counter(dr_single_author_ls),orient='index',columns=['count']).sort_values('count',ascending=False).head(20))

,count
Jensen M.,97
Fama E. F.,86
Black F.,63
Fama E.,62
R. Vishny,62
S. Titman,57
K. R. French,54
W. Meckling,52
H,49
권택호,45


,count
Fama E. F.,59
엄경식,43
S. Titman,37
[Provider:earticle],37
박경서,34
A. Shleifer,34
고봉찬,34
Fama Eugene F. Kenneth R. French,30
최혁,29
장하성,28


,count
Fama E. F.,364
K. R. French,287
Jensen M.,176
Black F.,149
W. Meckling,123
Carhart Mark M.,102
S. Titman,85
Barber B. M.,83
J. D. MacBeth,74
M. Scholes,73


,count
Fama E. F.,55
K. R. French,48
Black F.,41
D. Madan,34
Sutcliffe C.,33
윤선중,30
김솔,28
이재하,27
이우백,24
Bakshi G.,21
